In [1]:
from tlspt.datamodules.components.numpy_dataset import NumpyDataset
from tlspt.datamodules.components.base_site import BaseSiteDataset
from tlspt.datamodules.components.octree_dataset import OctreeDataset
from tlspt.datamodules.components.merged_dataset import MergedOctreeDataset

from tlspt.transforms import UniformTLSSampler
from tlspt.models.pointmae.pointmae import PointMAE
from torch.utils.data import DataLoader
import torch
from matplotlib import pyplot as plt

import numpy as np

In [2]:
seg_dataset = MergedOctreeDataset(
    split_files=['../data/plot_octrees/allen-spain/octrees/____TEST.csv'],
    split='train',
    scales=[1.5],
    min_points=[512],
    feature_names=['scalar_label'],
    features_to_normalize=None,
    normalize=True,
    transform=UniformTLSSampler(num_points=16384)
    )

seg_dataloader = DataLoader(seg_dataset, batch_size=8, shuffle=True)

2024-11-28 22:42:29.722 | INFO     | tlspt.datamodules.components.base_site:__init__:49 - OctreeDataset(../data/plot_octrees/allen-spain/octrees/____TEST.csv, train, 1.5): reading splits from ../data/plot_octrees/allen-spain/octrees/____TEST.csv
2024-11-28 22:42:29.745 | INFO     | tlspt.datamodules.components.base_site:__init__:56 - OctreeDataset(../data/plot_octrees/allen-spain/octrees/____TEST.csv, train, 1.5): looking for 1 folders in ../data/plot_octrees/allen-spain/octrees/
2024-11-28 22:42:29.748 | INFO     | tlspt.datamodules.components.base_site:__init__:64 - OctreeDataset(../data/plot_octrees/allen-spain/octrees/____TEST.csv, train, 1.5): found 1 plots for 'train' out of 1 plots defined in split file
2024-11-28 22:42:29.749 | INFO     | tlspt.structures.file_octree:__init__:155 - Initializing octree from <class 'str'> ../data/plot_octrees/allen-spain/octrees/SPA01_1,5m/SPA01.json
2024-11-28 22:42:29.914 | INFO     | tlspt.utils:__init__:37 - Creating directory for stats_file 

In [3]:
seg_dataset.prepare_data()

2024-11-28 22:42:30.526 | INFO     | tlspt.utils:prepare_data:53 - OctreeDataset.{'features_to_normalize': None}.train.33037d8734ce2: computing normalizer
  0% (0 of 1000) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--
  0% (1 of 1000) |                       | Elapsed Time: 0:00:00 ETA:   0:04:39
  0% (4 of 1000) |                       | Elapsed Time: 0:00:00 ETA:   0:01:55
  0% (5 of 1000) |                       | Elapsed Time: 0:00:00 ETA:   0:02:02
  0% (8 of 1000) |                       | Elapsed Time: 0:00:00 ETA:   0:01:34
  1% (13 of 1000) |                      | Elapsed Time: 0:00:00 ETA:   0:01:08
  1% (19 of 1000) |                      | Elapsed Time: 0:00:01 ETA:   0:00:53
  2% (24 of 1000) |                      | Elapsed Time: 0:00:01 ETA:   0:00:46
  3% (32 of 1000) |                      | Elapsed Time: 0:00:01 ETA:   0:00:40
  3% (35 of 1000) |                      | Elapsed Time: 0:00:01 ETA:   0:00:40
  3% (37 of 1000) |                      | El

In [4]:
seg_dataset[0]

{'points': tensor([[ 0.3107,  0.4886, -0.0087],
         [ 0.3006,  0.4742, -0.1146],
         [-0.1551, -0.5602, -0.0810],
         ...,
         [ 0.3181,  0.4927, -0.0129],
         [ 0.3847,  0.4480,  0.0232],
         [ 0.3540,  0.4502,  0.1653]]),
 'features': tensor([[1.],
         [1.],
         [0.],
         ...,
         [1.],
         [1.],
         [1.]]),
 'lengths': 3594}

In [ ]:
model = PointMAE(neighbor_alg='ball_query', ball_radius=0.2)